<a href="https://colab.research.google.com/github/qwertyqwertyqwertyqwerty123/colab_projects/blob/main/bs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

вариант 3 (наивный байес +3_laba_n2)


In [ ]:
# Для чтения xls
!pip install openpyxl

# Подключение к Google Диску
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/3laba_number_2.xlsx'

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Загрузка и обработка данных
df = pd.read_excel(file_path, header=None)
df = df.applymap(lambda x: float(str(x).replace(',', '.')))

X = df.iloc[:, :2].values
y = df.iloc[:, 2].values

# Разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Находим уникальные классы
classes = np.unique(y_train)

# Вычисляем среднее, дисперсию и априорные вероятности по каждому классу
class_stats = {}
for c in classes:
    X_c = X_train[y_train == c]
    mean = X_c.mean(axis=0)
    var = X_c.var(axis=0)
    prior = X_c.shape[0] / X_train.shape[0]
    class_stats[c] = {"mean": mean, "var": var, "prior": prior}

# Функция для вычисления вероятности по нормальному распределению
def gaussian_pdf(x, mean, var):
    eps = 1e-6  # для стабильности
    coef = 1.0 / np.sqrt(2 * np.pi * var + eps)
    exponent = np.exp(- ((x - mean) ** 2) / (2 * var + eps))
    return coef * exponent

# Функция для предсказания
def predict(X):
    preds = []
    for x in X:
        posteriors = []
        for c in classes:
            mean = class_stats[c]["mean"]
            var = class_stats[c]["var"]
            prior = class_stats[c]["prior"]
            likelihoods = gaussian_pdf(x, mean, var)
            posterior = np.prod(likelihoods) * prior
            posteriors.append(posterior)
        preds.append(classes[np.argmax(posteriors)])
    return np.array(preds)

# Предсказание и отчёт
y_pred = predict(X_test)
print(classification_report(y_test, y_pred, digits=4))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
              precision    recall  f1-score   support

         1.0     0.9167    0.9167    0.9167        72
         2.0     0.8120    0.8710    0.8405       124
         3.0     0.9444    0.8500    0.8947        40
         4.0     0.7966    0.7344    0.7642        64

    accuracy                         0.8500       300
   macro avg     0.8674    0.8430    0.8540       300
weighted avg     0.8515    0.8500    0.8497       300



<ipython-input-3-a4dffa7143f3>:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(str(x).replace(',', '.')))


Таким образом, мы получаем классификацию, в которой наибольшее значение precision и recall наблюдается для 1 класса; наименьшее - для 4-го.

